<a href="https://colab.research.google.com/github/ritamsk/santa_kaggle/blob/master/santa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import csv
import random
import sqlite3
import numpy as np
import  time
import pandas as pd
from scipy.optimize import linear_sum_assignment
import operator
nc=1000000

n_giftstype=1000
n_gift_of_each_type=1000
ng = n_gift_of_each_type*n_giftstype

triples=5000
twins=45000
others = triples+twins
nc1=nc-triples/3-twins/2

def ANCH(ChildHappiness, MaxChildHappiness, nc): #AverageNormalizedChildHappiness
    anch=0
    for i in ChildHappiness:
      anch += i/MaxChildHappiness
    anch = anch/nc
    return anch

def ANSH(GiftHappiness, MaxGiftHappiness, ng): #AverageNormalizedSantaHappiness
    ansh=0

    for i in GiftHappiness:
      ansh += i/MaxGiftHappiness
    ansh=ansh/ng
    return ansh

#need to maximaze
def ANH(ChildHappiness, GiftHappiness, MaxGiftHappiness, MaxChildHappiness, nc):   # Average Normalized Happiness
    anch = ANCH(ChildHappiness, MaxChildHappiness, nc)
    ansh = ANSH(GiftHappiness, MaxGiftHappiness, ng)
    anh = anch**3 + ansh**3
    print(anch, ansh, 'anh', anch+ansh)
    return anh
  
  
def sort_dict_by_values(a, reverse=True):
    sorted_x = sorted(a.items(), key=operator.itemgetter(1), reverse=True)
    return sorted_x



print('start')
MaxChildHappiness = 100 * 2.
MaxGiftHappiness = 1000 * 2.

f = open('submission.csv', 'w', newline='', encoding='utf-8')
awriter = csv.writer(f)

mainstart = time.clock()
gifts_wishlist=pd.read_csv('gift_goodkids_v2.csv', header=None).as_matrix()[:, 1:]
end = time.clock()
print('reading gift_goodkids, time: ', (end-mainstart)/60)
gifts_happiness = 0.*np.ones((n_giftstype, nc), dtype=np.float32)
print('gifts_happiness')
for g in range(n_giftstype):
    for i, c in enumerate(gifts_wishlist[g]):
        gifts_happiness[g,c] += (2.*(1000 - i))
        

del gifts_wishlist

start = time.clock()
childs_wishlist=pd.read_csv('child_wishlist_v2.csv', header=None).as_matrix()[:, 1:]
end = time.clock()
print('reading child_goodkids, time: ', (end-start)/60)


print('childs_happiness')
childs_happiness = 0.*np.ones((nc, n_giftstype), dtype=np.float32)
for c in range(nc):
    for i, g in enumerate(childs_wishlist[c]):
        childs_happiness[c,g] +=(2. * (100 - i) )
        
del childs_wishlist

ChildHappiness = []
GiftHappiness = [] 


gifts_id = np.full((n_giftstype), 0)

answer = np.full((nc, 2), -1)
for i in range(0, nc):
  answer[i, 0]=i

while sum(gifts_id)<nc:
    print(time.ctime(time.time()))
    gifts =  np.where(gifts_id < 1000)
    for g in gifts[0]:
        print('sum', sum(gifts_id))
        # print( np.where(childs_happiness[:, g]>-1.)[0])
        gift_happiness = gifts_happiness[g, :]
        child_happiness = childs_happiness[:, g] 
        child_happiness = child_happiness + gift_happiness
        childs = np.where(child_happiness > 0)
        child_dict = dict(zip(childs[0], child_happiness[childs[0]]))
        child_dict = sort_dict_by_values(child_dict)
  
        childs=[]
        for i in range(0, len(child_dict)):
            childs.append(child_dict[i][0])
          
        for c in childs:
          
            if gifts_id[g] < 998 and c < triples - 1:
                if c%3==0:
                    if answer[c, 1] == -1:
                            answer[c ,  1] = g
                            answer[c +1, 1] = g
                            answer[c+2, 1] = g
                            gifts_id[g] += 3

                    else:
                        if (childs_happiness[c, g] + gifts_happiness[g, c] + childs_happiness[c+1, g] + gifts_happiness[g, c+1] + childs_happiness[c +2, g] + gifts_happiness[g, c +2])  > (childs_happiness[c, answer[c, 1]] + gifts_happiness[answer[c, 1], c] + childs_happiness[c+1, answer[c+1, 1]] + gifts_happiness[answer[c+1, 1], c+1] + childs_happiness[c - 1, answer[c +2, 1]] + gifts_happiness[answer[c+2, 1], c +2]):

                            gifts_id[answer[c+1, 1]] -= 1
                            gifts_id[answer[c+2, 1]] -= 1
                            gifts_id[answer[c, 1]] -= 1
                            answer[c, 1] = g
                            answer[c+1, 1] = g
                            answer[c+2, 1] = g
                            gifts_id[g] += 3


                elif (c+1)%3==0:
                    if answer[c, 1] == -1:
                          answer[c -1, 1] = g
                          answer[c -2, 1] = g
                          answer[c, 1] = g
                          gifts_id[g] += 3

                    else:
                        if (childs_happiness[c, g] + gifts_happiness[g, c] + childs_happiness[c-2, g] + gifts_happiness[g, c-2] + childs_happiness[c - 1, g] + gifts_happiness[g, c -1] ) >   (childs_happiness[c, answer[c, 1]] + gifts_happiness[answer[c, 1], c] + childs_happiness[c+1, answer[c-2, 1]] + gifts_happiness[answer[c-2, 1], c-2] + childs_happiness[c - 1, answer[c - 1, 1]] + gifts_happiness[answer[c -1, 1], c - 1] ):

                            gifts_id[answer[c, 1]] -= 1
                            gifts_id[answer[c-2, 1]] -= 1
                            gifts_id[answer[c - 1, 1]] -= 1
                            answer[c-1, 1] = g
                            answer[c-2, 1] = g
                            answer[c, 1] = g
                            gifts_id[g] += 3


                elif (c - 1) % 3 == 0:
                    if answer[c, 1] == -1:
                          answer[c - 1, 1] = g
                          answer[c, 1] = g
                          answer[c +1, 1] = g
                          gifts_id[g] += 3

                    else:
                        if (childs_happiness[c, g] + gifts_happiness[g, c] + childs_happiness[c+1, g] + gifts_happiness[g, c+1] + childs_happiness[c - 1, g] + gifts_happiness[g, c -1]) > (childs_happiness[c, answer[c, 1]] + gifts_happiness[answer[c, 1], c] + childs_happiness[c+1, answer[c+1, 1]] + gifts_happiness[answer[c+1, 1], c+1] + childs_happiness[c - 1, answer[c - 1, 1]] + gifts_happiness[answer[c -1, 1], c - 1]):

                              gifts_id[answer[c-1, 1]] -= 1
                              gifts_id[answer[c, 1]] -= 1
                              gifts_id[answer[c +1, 1]] -= 1
                              answer[c-1, 1] = g
                              answer[c, 1] = g
                              answer[c+1, 1] = g
                              gifts_id[g] += 3


            elif gifts_id[g] < 999 and c > triples and c < twins:               
                if c % 2 == 0:                  
                    if answer[c, 1] == -1:
                          answer[c, 1] = g
                          answer[c - 1, 1] = g
                          gifts_id[g] += 2
                    else:
                        if (childs_happiness[c, g] + gifts_happiness[g, c] + childs_happiness[c - 1, g] + gifts_happiness[g, c - 1]) > (childs_happiness[c, answer[c, 1]] + gifts_happiness[answer[c, 1], c] + childs_happiness[c - 1, answer[c - 1, 1]] + gifts_happiness[answer[c - 1, 1], c - 1]):
                            gifts_id[answer[c, 1]] -= 1
                            gifts_id[answer[c - 1, 1]] -= 1
                            answer[c, 1] = g
                            answer[c - 1, 1] = g
                            gifts_id[g] += 2
                else :
                    if answer[c, 1] == -1:
                          answer[c + 1, 1] = g
                          answer[c , 1] = g
                          gifts_id[g] += 2
                    else:
                        if (childs_happiness[c, g] + gifts_happiness[g, c] + childs_happiness[c - 1, g] + gifts_happiness[g, c + 1]) > (childs_happiness[c, answer[c, 1]] + gifts_happiness[answer[c, 1], c] + childs_happiness[c + 1, answer[c + 1, 1]] + gifts_happiness[answer[c - 1, 1], c + 1]):
                            gifts_id[answer[c, 1]] -= 1
                            gifts_id[answer[c - 1, 1]] -= 1
                            answer[c + 1, 1] = g
                            answer[c, 1] = g
                            gifts_id[g] += 2
                            
            elif gifts_id[g] < 1000 and c > twins:
                if answer[c, 1] == -1:
                    answer[c, 1] = g
                    gifts_id[g] += 1
                else:
                    if (childs_happiness[c, g] + gifts_happiness[g, c]) > (childs_happiness[c, answer[c, 1]] + gifts_happiness[answer[c, 1], c]):
                            gifts_id[answer[c, 1]] -= 1
                            answer[c, 1] = g
                            gifts_id[g] += 1
        ChildHappiness = []
        GiftHappiness = [] 
        if (sum(gifts_id) >910000 and sum(gifts_id)<910500) or (sum(gifts_id) >990000 and sum(gifts_id)<990500):
            for c, g  in answer:
                if g!=-1:
                  ChildHappiness.append(childs_happiness[c, g])
                  GiftHappiness.append(gifts_happiness[g, c])

            anh = ANH(ChildHappiness, GiftHappiness, MaxGiftHappiness, MaxChildHappiness,  sum(gifts_id))
            print("anh",anh)
            f = open('submission.csv', 'w', newline='', encoding='utf-8')
            awriter = csv.writer(f)
            for row in answer:
                row_csv=row
                awriter.writerow(row_csv)
            f.close()
           
           
          
              
            
ChildHappiness = []
GiftHappiness = [] 
for c, g  in answer:
  if g!=-1:
    ChildHappiness.append(childs_happiness[c, g])
    GiftHappiness.append(gifts_happiness[g, c])

anh = ANH(ChildHappiness, GiftHappiness, MaxGiftHappiness, MaxChildHappiness, nc)

print("anh",anh)
print('answer', answer)
print('gifts', gifts_id)
#answer = np.concatenate([answer_triples, answer_twins, answer])
mainend = time.clock()

print(time.ctime(time.time()))

print('writing csv')

for row in answer:
    row_csv=row
    awriter.writerow(row_csv)


print('main time: ', (mainend-start)/60)
print("All time: ", (mainend-mainstart)/60)



sum 968581
sum 968581
sum 968588
sum 968589
sum 968590
sum 968591
sum 968592
sum 968592
sum 968593
sum 968597
sum 968603
Fri Dec 21 23:51:13 2018
sum 968603
sum 968603
sum 968604
sum 968606
sum 968606
sum 968607
sum 968607
sum 968607
sum 968608
sum 968608
sum 968608
sum 968608
sum 968609
sum 968610
sum 968610
sum 968610
sum 968610
sum 968610
sum 968616
sum 968616
sum 968616
sum 968616
sum 968617
sum 968617
sum 968618
sum 968618
sum 968623
sum 968623
sum 968623
sum 968623
sum 968623
sum 968625
sum 968625
sum 968626
sum 968627
sum 968627
sum 968631
sum 968631
sum 968631
sum 968631
sum 968632
sum 968632
sum 968632
sum 968634
sum 968635
sum 968635
sum 968635
sum 968635
sum 968635
sum 968635
sum 968636
sum 968637
sum 968637
sum 968637
sum 968640
sum 968640
sum 968641
sum 968641
sum 968641
sum 968642
sum 968642
sum 968643
sum 968646
sum 968647
sum 968647
sum 968647
sum 968647
sum 968647
sum 968648
sum 968654
sum 968654
sum 968654
sum 968654
sum 968654
sum 968654
sum 968654
sum 968655
sum 968